In [36]:
from Bio.Seq import Seq
import pandas as pd
import os
import math
import datetime

In [37]:
"""
Input: a txt file containing gRNA sequences. 
    The file can be either one column of gRNA sequences only, or the output file from 1_SNP_gRNA

Output: a txt file containing primers that assemble gRNAs into the CREEPY vector
"""

'\nInput: a txt file containing gRNA sequences. The file can be either one column of gRNA sequences only, or the output file from 1_SNP_gRNA\n\nOutput: a txt file containing primers that assemble gRNAs into the CREEPY vector\n'

In [38]:
#Input filename
gRNA_FILENAME = "pXW470_gRNA_FL.txt"

#Output filename (partial)
creepy_primer_file = "pXW470_creepy_primer_" 

desktop = os.path.expanduser("~/Desktop")
gRNA_file_path = desktop + '/' + gRNA_FILENAME


In [39]:
def read_data(file_path):
    try:
        df = pd.read_csv(file_path, sep=None)
        if "SNP" in df.columns and "gRNA" in df.columns:
            df["SNP"] = df["SNP"].astype(str)
            df["gRNA"] = df["gRNA"].str.replace(" ", "")
            df["gRNA"] = df["gRNA"].apply(lambda seq_str: Seq(seq_str.upper()))
            return df
    except pd.errors.EmptyDataError:
        pass

    try:
        df = pd.read_csv(file_path, header=None, names=["gRNA"])
        df['SNP'] = range(1, df.shape[0]+1)
        df['SNP'] = df['SNP'].astype(str)
        df["gRNA"] = df["gRNA"].str.replace(" ", "")
        df["gRNA"] = df["gRNA"].apply(lambda seq_str: Seq(seq_str.upper()))
        return df
    except FileNotFoundError:
        pass

    return None

In [41]:
gRNA_df = read_data(gRNA_file_path)
gRNA_df

C:\Users\wxh02\AppData\Local\Temp\ipykernel_24636\3556184501.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(file_path, sep=None)


,SNP,gRNA
0,rs2239017,"(A, G, A, A, A, G, T, G, A, G, T, C, A, T, T, ..."
1,rs758171,"(A, C, C, A, T, G, C, T, G, T, G, G, C, C, A, ..."
2,rs71441679,"(A, A, G, G, C, T, G, A, G, C, A, T, C, T, G, ..."
3,rs35848523,"(T, A, T, T, C, T, G, A, T, A, G, A, T, A, A, ..."
4,rs2238049,"(T, G, A, A, G, C, C, T, C, T, A, G, G, G, C, ..."
5,rs2238050,"(G, A, A, A, A, A, T, T, A, C, T, A, G, C, C, ..."
6,rs2238051,"(C, C, G, G, T, A, G, T, A, A, A, A, A, T, T, ..."
7,rs1860056,"(G, C, C, A, C, C, A, C, T, A, A, C, A, G, C, ..."
8,rs12423277,"(T, G, G, C, T, T, C, C, T, T, C, T, A, G, A, ..."
9,rs1016388,"(T, A, C, A, C, A, G, A, G, G, A, A, A, G, A, ..."


In [48]:
def creepy_primer(df, hf="Medium"):
    """
    Design CREEPY primers based on a list of gRNAs
    For one guide RNA, return two oligos that generate sticky ends after annealing.
    For two guide RNAs, return two primers, with each primer incorperating one gRNA.
    For three and more guide RNAs, return a set of primers:
        The first and last primers introduce the first and last gRNAs, respectively.
        The rest of the primers introduce the middle gRNAs that are each split into two halves,
        with the first half in the reverse primer and the second half in the fwd primer.

    :param 
    df: dataframe containing the SNPs and gRNAs
    hf: High-fidelity boolean. default is medium.
        False: do not check ligation fidelity dictionary
        low: low fidelity, only avoid highly frequent misligations (freq>20%)
        medium: medium fidelity, avoid misligation freq>10%
        high: high fidelity, avoid misligation freq>1%

    :return: 
    a dictionary of primers, and a dictionary of sticky end sequences
    """
    esp3i_adaptor = Seq('aCGTCTCc')   #universal adaptor for Esp3I, will generate a sticky end that matches the 4 following nt
    sticky_end_left = Seq('GACT')    #left sticky end of pXW467/468/472
    sticky_end_right = Seq('AAAC')   #right sticky end of pXW67/468/472
    template_fwd = Seq('GTTTTAGAGCTAGAAATAGCAAGTTA') #Fwd primer to amplify scaffold RNA
    template_rev = Seq('TGCGCAAGCCCGGAATCGAACCGGG')  #Rev primer to amplify tRNA_Gly
    adaptor_left = esp3i_adaptor + sticky_end_left + "tt" #left adaptor for Esp3I to generate a GACT sticky end followed by tt
    adaptor_right = esp3i_adaptor + sticky_end_right  #Right adaptor for Esp3I to generate a AAAC sticky end
    primer_dict = {}
    overhang_dict = {"vect_left": str(sticky_end_left) + "\tNA", "vect_right": str(sticky_end_right) + "\tNA"}
    overhang_fidelity_dict = {}

    if df.shape[0] == 1:
        # For one gRNA only
        primer1_name, primer1 = df['SNP'].iloc[0] + '_F', sticky_end_left + 'tt' + df['gRNA'][0]
        primer2_name, primer2 = df['SNP'].iloc[0] + '_R', sticky_end_right + df['gRNA'][0].reverse_complement() + 'aa'
        primer_dict = {primer1_name: primer1, primer2_name: primer2}
        overhang_fidelity_dict["vect_left"], overhang_fidelity_dict["vect_right"] = "NA", "NA"

    elif df.shape[0] == 2:
        # For two gRNAs
        primer1_name, primer1 = df['SNP'].iloc[0] + '_F', adaptor_left + df['gRNA'][0] + template_fwd
        primer2_name, primer2 = df['SNP'].iloc[1] + '_R', adaptor_right + df['gRNA'][1].reverse_complement() + template_rev
        primer_dict = {primer1_name: primer1, primer2_name: primer2}
        overhang_fidelity_dict["vect_left"], overhang_fidelity_dict["vect_right"] = "NA", "NA"

    else:
        # For three and more gRNAs (1st and last are introduced in one primer, all others are split into two primers and joined by golden gate
        overhang_start_list = []
        primer1_name = df['SNP'].iloc[0] + '_F'
        primer_dict[primer1_name] = adaptor_left + df['gRNA'][0] + template_fwd
        overhang_list = [str(sticky_end_left), str(sticky_end_right)]   

        if hf:
            ligation_fidelity_dict = "Ligation_fidelity_dictionary.json"
            hf_combined = pd.read_json(ligation_fidelity_dict)
            hf_df = hf_combined[hf.lower()]

            try: overhang_fidelity_dict = {"vect_left": hf_df[sticky_end_left], "vect_right": hf_df[sticky_end_right]}
            except KeyError:
                pass

            try: overhang_list.extend(hf_df[sticky_end_left])
            except KeyError:
                pass

            try: overhang_list.extend(hf_df[sticky_end_right])
            except KeyError:
                pass            

            for n in range(1,len(df['gRNA'])-1):
                gRNA_seq = df['gRNA'][n]
                overhang_start = 8
                overhang = gRNA_seq[overhang_start:overhang_start + 4]
                i=1

                while overhang.reverse_complement() == overhang or overhang in overhang_list or overhang.reverse_complement() in overhang_list:
                    overhang_start = 8 + math.ceil(i/2) * (-1) ** i
                    overhang = gRNA_seq[overhang_start:overhang_start+4]
                    i = i + 1

                try: overhang_misligate = hf_df[overhang]
                except KeyError:
                    pass   

                overhang_list.append(str(overhang))
                overhang_list.extend(overhang_misligate)

                overhang_dict[df['SNP'][n]] = f"{str(overhang)}\t{overhang_start+1}"
                p_r_name = df['SNP'].iloc[n] + "_aR"
                p_f_name = df['SNP'].iloc[n] + "_bF"
                primer_dict[p_r_name] = esp3i_adaptor + gRNA_seq[:overhang_start+4].reverse_complement() + template_rev
                primer_dict[p_f_name] = esp3i_adaptor + gRNA_seq[overhang_start:] + template_fwd

                overhang_fidelity_dict[df['SNP'][n]] = overhang_misligate

        else:
            for n in range(1,len(df['gRNA'])-1):
                gRNA_seq = df['gRNA'][n]
                overhang_start = 8
                overhang = gRNA_seq[overhang_start:overhang_start + 4]
                i=1

                while overhang.reverse_complement() == overhang or overhang in overhang_list or overhang.reverse_complement() in overhang_list:
                    overhang_start = 8 + math.ceil(i/2) * (-1) ** i
                    overhang = gRNA_seq[overhang_start:overhang_start+4]
                    i = i + 1
                overhang_list.append(str(overhang))

                overhang_dict[df['SNP'][n]] = f"{str(overhang)}\t{overhang_start+1}"
                p_r_name = df['SNP'].iloc[n] + "_aR"
                p_f_name = df['SNP'].iloc[n] + "_bF"
                primer_dict[p_r_name] = esp3i_adaptor + gRNA_seq[:overhang_start+4].reverse_complement() + template_rev
                primer_dict[p_f_name] = esp3i_adaptor + gRNA_seq[overhang_start:] + template_fwd
                overhang_fidelity_dict[df['SNP'][n]] = "NA"
            overhang_fidelity_dict["vect_left"], overhang_fidelity_dict["vect_right"] = "NA", "NA"


        # overhang_fidelity_dict["vect_left"], overhang_fidelity_dict["vect_right"] = "NA", "NA"

        primer_last_name = df['SNP'].iloc[-1] + "_R"
        primer_dict[primer_last_name] = adaptor_right + df['gRNA'].iloc[-1].reverse_complement() + template_rev
        print(overhang_list)

    return  primer_dict, overhang_dict, overhang_fidelity_dict


In [50]:
primer_set, sticky_end_dict, fidelity_dict = creepy_primer(gRNA_df,hf="high")

current_datetime = datetime.datetime.now()
date_time_str = current_datetime.strftime("%Y-%m-%d_%H-%M-%S")

creepy_primer_path = desktop + '/' + creepy_primer_file + date_time_str + ".txt"

with open(creepy_primer_path, "w") as file:
    file.write('Primer\t\tSequence\n')
    for key, value in primer_set.items():
        file.write(f"{key}\t{value}\n")
    file.write('\nSNP\t\tStk_end\tPos\tIncompatible\n')
    for key, value in sticky_end_dict.items():
        file.write(f"{key}\t{value}\t{fidelity_dict[key]}\n")


['GACT', 'AAAC', 'AGTT', 'TGTC', 'GGTC', 'AGGC', 'AGCC', 'AGTA', 'GTTG', 'GTGG', 'CCAT', 'CCTC', 'CCGC', 'CTAC', 'CGAC', 'TCAC', 'GCAC', 'ACAC', 'GCAT', 'ATGT', 'TTGC', 'GTGC', 'AGGC', 'TAGA', 'TCTT', 'TCTG', 'TAGG', 'CCTT', 'CCTG', 'CCTC', 'TCTA', 'ACTA', 'TGGT', 'AAAA', 'TGGT', 'TAAC', 'GTTT', 'GTTG', 'GTTC', 'TTCT', 'AGAT', 'AGAG', 'GGAA', 'TTGC', 'CTGC', 'GCTG', 'GCGG', 'GCCG', 'GTAG', 'GGAG', 'ACAG', 'ACTG', 'CTGT', 'CGGT', 'TAGT', 'CAGG', 'CAGC', 'GAAG', 'CTTT', 'CTTG', 'TTTC', 'CTAC', 'CTTA', 'GAGA', 'GAGC', 'GCTT', 'GCTG', 'GTTC', 'GGTC', 'ACTC', 'GATC', 'GCCC', 'GCAC', 'GCTA', 'TGAC', 'GTCT', 'GTCG', 'GTCC', 'GTTA', 'GGCA', 'CAAA', 'ATGA']


In [24]:
# def creepy_primer(df, hf="medium"):
#     """
#     Design CREEPY primers based on a list of gRNAs
#     For one guide RNA, return two oligos that generate sticky ends after annealing.
#     For two guide RNAs, return two primers, with each primer incorperating one gRNA.
#     For three and more guide RNAs, return a set of primers:
#         The first and last primers introduce the first and last gRNAs, respectively.
#         The rest of the primers introduce the middle gRNAs that are each split into two halves,
#         with the first half in the reverse primer and the second half in the fwd primer.
#         
#     :param 
#     df: dataframe containing the SNPs and gRNAs
#     hf: High-fidelity boolean. default is medium.
#         False: do not check ligation fidelity dictionary
#         low: low fidelity, only avoid highly frequent misligations (freq>20%)
#         medium: medium fidelity, misligation freq>10%
#         high: high fidelity, avoid misligation freq>1%
#     
#     :return: 
#     a dictionary of primers, and a dictionary of sticky end sequences
#     """
#     esp3i_adaptor = Seq('aCGTCTCc')   #universal adaptor for Esp3I, will generate a sticky end that matches the 4 following nt
#     sticky_end_left = Seq('GACT')    #left sticky end of pXW467/468/472
#     sticky_end_right = Seq('AAAC')   #right sticky end of pXW67/468/472
#     template_fwd = Seq('GTTTTAGAGCTAGAAATAGCAAGTTA') #Fwd primer to amplify scaffold RNA
#     template_rev = Seq('TGCGCAAGCCCGGAATCGAACCGGG')  #Rev primer to amplify tRNA_Gly
#     adaptor_left = esp3i_adaptor + sticky_end_left + "tt" #left adaptor for Esp3I to generate a GACT sticky end followed by tt
#     adaptor_right = esp3i_adaptor + sticky_end_right  #Right adaptor for Esp3I to generate a AAAC sticky end
#     primer_dict = {}
#     overhang_dict = [{"SNP": "vect_left", "sticky_end": sticky_end_left, "position": "NA", "Fidelity": "NA"}, 
#                     {"SNP": "vect_right", "sticky_end": sticky_end_right, "position": "NA", "Fidelity": "NA"}]
#     overhang_df = pd.DataFrame(overhang_dict)
#     
#     if df.shape[0] == 1:
#         # For one gRNA only
#         primer1_name, primer1 = df['SNP'].iloc[0] + '_F', sticky_end_left + 'tt' + df['gRNA'][0]
#         primer2_name, primer2 = df['SNP'].iloc[0] + '_R', sticky_end_right + df['gRNA'][0].reverse_complement() + 'aa'
#         primer_dict = {primer1_name: primer1, primer2_name: primer2}
#         
#     elif df.shape[0] == 2:
#         # For two gRNAs
#         primer1_name, primer1 = df['SNP'].iloc[0] + '_F', adaptor_left + df['gRNA'][0] + template_fwd
#         primer2_name, primer2 = df['SNP'].iloc[1] + '_R', adaptor_right + df['gRNA'][1].reverse_complement() + template_rev
#         primer_dict = {primer1_name: primer1, primer2_name: primer2}
#         
#     else:
#         # For three and more gRNAs (1st and last are introduced in one primer, all others are split into two primers and joined by golden gate
#         overhang_start_list = []
#         primer1_name = df['SNP'].iloc[0] + '_F'
#         primer_dict[primer1_name] = adaptor_left + df['gRNA'][0] + template_fwd
#         overhang_list = [str(sticky_end_left), str(sticky_end_right)]   
#         
#         if hf:
#             ligation_fidelity_dict = "Ligation_fidelity_dictionary.json"
#             hf_combined = pd.read_json(ligation_fidelity_dict)
#             hf_df = hf_combined[hf]
#                      
#             try: overhang_df["Fidelity"]["vect_left"], overhang_df["Fidelity"]["vect_right"] = hf_df[sticky_end_left], hf_df[sticky_end_right]
#             except KeyError:
#                 pass      
#             
#             for n in range(1,len(df['gRNA'])-1):
#                 gRNA_seq = df['gRNA'][n]
#                 overhang_start = 8
#                 overhang = gRNA_seq[overhang_start:overhang_start + 4]
#                 i=1
#                          
#                 while overhang.reverse_complement() == overhang \
#                     or any(overhang in row for row in overhang_df['sticky_end']) \
#                     or any(overhang.reverse_complement() in row for row in overhang_df['sticky_end']) \
#                     or any(str(overhang) in row for row in overhang_df['Fidelity']) \
#                     or any(str(overhang.reverse_complement()) in row for row in overhang_df['Fidelity']):
#                     overhang_start = 8 + math.ceil(i/2) * (-1) ** i
#                     overhang = gRNA_seq[overhang_start:overhang_start+4]
#                     i = i + 1
#                 
#                 try: new_row = pd.Series({"SNP": df['SNP'][n], "sticky_end": str(overhang), "position": overhang_start+1, "Fidelity": hf_df[overhang]})
#                 except KeyError:
#                     pass   
#                 overhang_df = pd.concat([overhang_df, new_row])    
#                             
#                 # overhang_dict[df['SNP'][n]] = f"{str(overhang)} start_pos: ({overhang_start+1})"
#                 p_r_name = df['SNP'].iloc[n] + "_aR"
#                 p_f_name = df['SNP'].iloc[n] + "_bF"
#                 primer_dict[p_r_name] = esp3i_adaptor + gRNA_seq[:overhang_start+4].reverse_complement() + template_rev
#                 primer_dict[p_f_name] = esp3i_adaptor + gRNA_seq[overhang_start:] + template_fwd
#             
#         else:
#             for n in range(1,len(df['gRNA'])-1):
#                 gRNA_seq = df['gRNA'][n]
#                 overhang_start = 8
#                 overhang = gRNA_seq[overhang_start:overhang_start + 4]
#                 i=1
#                 
#                 while overhang.reverse_complement() == overhang \
#                     or any(overhang in row for row in overhang_df['sticky_end']) \
#                     or any(overhang.reverse_complement() in row for row in overhang_df['sticky_end']):
#                     i = i + 1
#            
#                 p_r_name = df['SNP'].iloc[n] + "_aR"
#                 p_f_name = df['SNP'].iloc[n] + "_bF"
#                 primer_dict[p_r_name] = esp3i_adaptor + gRNA_seq[:overhang_start+4].reverse_complement() + template_rev
#                 primer_dict[p_f_name] = esp3i_adaptor + gRNA_seq[overhang_start:] + template_fwd
#                 
#                 new_row = pd.Series({"SNP": df['SNP'][n], "sticky_end": str(overhang), "position": overhang_start+1, "Fidelity": "NA"})
#                 overhang_df = pd.concat([overhang_df, new_row])
#                 
#         primer_last_name = df['SNP'].iloc[-1] + "_R"
#         primer_dict[primer_last_name] = adaptor_right + df['gRNA'].iloc[-1].reverse_complement() + template_rev
#         print(overhang_list)
#     
#     return  primer_dict, overhang_df
